# Before Start!

first start with connecting the local runtime.

In [1]:
!docker ps -a 

CONTAINER ID        IMAGE                             COMMAND                  CREATED             STATUS                     PORTS               NAMES
8a555e709777        sakha002/pyprotobuf2:latest       "/bin/bash"              10 days ago         Exited (0) 9 days ago                          determined_williamson
03d84bc5c6a9        sakha002/historyserver:latest     "/entrypoint.sh /run…"   2 weeks ago         Exited (137) 2 weeks ago                       historyserver
7f21947fc924        sakha002/nodemanager:latest       "/entrypoint.sh /run…"   2 weeks ago         Exited (137) 2 weeks ago                       nodemanager1
879d3c554959        sakha002/resourcemanager:latest   "/entrypoint.sh /run…"   2 weeks ago         Exited (137) 2 weeks ago                       resourcemanager
05f6f655c88d        sakha002/datanode:latest          "/entrypoint.sh /run…"   2 weeks ago         Exited (137) 2 weeks ago                       datanode3
70402d515bdb        sakha002/datanode:lates

okay some clean up of containers
Q:  when a container exits, can we resume it and continue where we left off? should be ..

In [4]:
!docker container prune 

unknown shorthand flag: 'y' in -y
See 'docker container prune --help'.


In [5]:
!docker container prune -f

Deleted Containers:
8a555e7097775839b8d6a3ddd31048b73b8ff94435e793dff50a69d5f7526a12
03d84bc5c6a9ee97c43e2eba85b855c356745991f93d7e7b82dce6026713d5a9
7f21947fc924301c495848071d2606673d68b686cef47bea445cedacaad8d256
879d3c5549594c5064a2a75d719acf10932186306cdf9c602706fb333816ad15
05f6f655c88ddb5e25dce55f0fe0247aca2a5e7559dafbad90c03cf75e868d7a
70402d515bdb0af0a780092119253d91ab898b4176baa9ce865e7acc067b7ba5
d0da94ad0f4a03a49e5e13d2930b7a8af1c0765ea1666a2bc1a6290a8256049c
8a73b54e28e53ec30a278ed799b91979160c26c49b8a9be451dc2ccd35ac5081
531e654272af1052a125b6f764d5a34d7e1df1f219c7b274dcaa0c9a248a96a5
8641e0a64e033f301dfc2f565cd3bd0d9ebf87e042ca9c9a2131b002c04bc3cc
f6e8281f6f27dcb86cc7190d4d3be63067fdb510fa1965033d649389b3a07c24
d2352454e66f5619f387c287f84bd0b09d59a775af33a5609c5e671987c51ae8
bb07ab843c4d0d10921f9c20fcf2aa7c9fa8a8bfbd23274763ec8b9abebb0629
4493421d5a28a6907371b45ca42f18c9116304373a35f2b38f920f8cdba339a8
658bed8ba63f7fe734c94b46acd68f4dbe0a3578bd69058a8d1aa47330992084
11c58

In [6]:
!docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


Now getting back to the hadoop containers. 
where are they?

In [7]:
!ls

Desktop    Downloads  localperl  Pictures  snap       Videos
Documents  gopath     Music	 Public    Templates


In [8]:
%cd ../../media/hossein/HDD/gdrive/Active_projects/my_dfs/
!ls

/media/hossein/HDD/gdrive/Active_projects/my_dfs
docker-hadoop				     protobuf
hadoop-mapreduce-examples-2.7.1-sources.jar  protobuf-python-3.13.0.tar.gz
hadoop_source				     python_hadoop_examples


In [9]:
%cd ../../SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs/
!ls


/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs
docker-hadoop				     hadoop_source
hadoop-mapreduce-examples-2.7.1-sources.jar  python_hadoop_examples


In [10]:
%cd docker-hadoop/
!ls

/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs/docker-hadoop
base		       docker-compose.yml  Makefile  nodemanager      submit
datanode	       hadoop.env	   namenode  README.md
docker-compose-v3.yml  historyserver	   nginx     resourcemanager


In [12]:
!docker-compose up -d

Creating namenode ... 
Creating datanode3 ... 
Creating datanode2 ... 
Creating datanode1 ... 
Creating resourcemanager ... 
Creating historyserver   ... 
Creating nodemanager1    ... 


okay so observation is that using this docker compose fails to properly spin up the resource manager container everytime, and I will end up doing it manually (using the vscode UI)
but now I guess we are generally all set for the streaming.


# Q: how to work with the hadoop dfs
 (should get this figured out)

# Back to Streaming Examples

okay so in the previous example, I did that well worn word count using the map reduce scripts.

Should I remind myself of what was the map part, and what was the reduce part?
okay so I guess there was some more falvors of the same word count example that I can try and refresh mind too.




##WordCount Example (reminers)



```
hduser@ubuntu:/usr/local/hadoop$ bin/hadoop dfs -copyFromLocal /tmp/gutenberg /user/hduser/gutenberg
hduser@ubuntu:/usr/local/hadoop$ bin/hadoop dfs -ls
```

we let the **Hadoop group the (key, value) pairs** between the Map and the Reduce step because Hadoop is more efficient in this regard than our simple Python scripts.

okay now we add the second mapper and reducrs for the Word count:

```
#!/usr/bin/env python3
"""A more advanced Mapper, using Python iterators and generators."""

import sys

def read_input(file):
    for line in file:
        # split the line into words
        yield line.split()

def main(separator='\t'):
    # input comes from STDIN (standard input)
    data = read_input(sys.stdin)
    for words in data:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        for word in words:
            print ('%s%s%d' % (word, separator, 1))

if __name__ == "__main__":
    main()
```


```
#!/usr/bin/env python
"""A more advanced Reducer, using Python iterators and generators."""

from itertools import groupby
from operator import itemgetter
import sys

def read_mapper_output(file, separator='\t'):
    for line in file:
        yield line.rstrip().split(separator, 1)

def main(separator='\t'):
    # input comes from STDIN (standard input)
    data = read_mapper_output(sys.stdin, separator=separator)
    # groupby groups multiple word-count pairs by word,
    # and creates an iterator that returns consecutive keys and their group:
    #   current_word - string containing a word (the key)
    #   group - iterator yielding all ["&lt;current_word&gt;", "&lt;count&gt;"] items
    for current_word, group in groupby(data, itemgetter(0)):
        try:
            total_count = sum(int(count) for current_word, count in group)
            print "%s%s%d" % (current_word, separator, total_count)
        except ValueError:
            # count was not a number, so silently discard this item
            pass

if __name__ == "__main__":
    main()

```


### summary of my take on mapper and reducer





okay so now we have our mapper and reducer.
so a quick look at them shows that  

* the mapper reads the  Data from **stdin**  and generates a set of (key, values)  in **stdout** ( which is print )

* the hadoop  groups this  (big) set by Key values ( or maybe something else, hmmm!)

* the reducer gets the  set of  (key,value)s  from  **stdin** .  It performs some grouping based on the keys ( and maybe some computation of the values) and then produce the results for their Set to the **stdout**

okay 

### Running the mapper/reducer with streaming


okay checked inside my hadoop cluster name node. we have python3 there , though the version is old (3.5.3) but anyway!

The Streaming jar file is not in there.

I need to copy that file plus the mapper/reducer files into the name node


Also the books and all the other stuff that I have added to the hdfs before is there still.
```
root@ca146de6e692:/# hdfs dfs -ls
Found 15 items
drwxr-xr-x   - root supergroup          0 2020-10-08 20:30 daily
drwxr-xr-x   - root supergroup          0 2020-10-11 02:14 gutenberg
drwxr-xr-x   - root supergroup          0 2020-10-10 17:16 gutenberg-output
drwxr-xr-x   - root supergroup          0 2020-10-11 02:19 gutenberg-output11
drwxr-xr-x   - root supergroup          0 2020-10-11 02:33 gutenberg-output12
drwxr-xr-x   - root supergroup          0 2020-10-10 17:42 gutenberg-output2
drwxr-xr-x   - root supergroup          0 2020-10-10 17:46 gutenberg-output3
drwxr-xr-x   - root supergroup          0 2020-10-11 02:15 gutenberg-output4
drwxr-xr-x   - root supergroup          0 2020-10-11 01:23 gutenberg-output5
drwxr-xr-x   - root supergroup          0 2020-10-11 01:33 gutenberg-output6
drwxr-xr-x   - root supergroup          0 2020-10-11 01:38 gutenberg-output7
drwxr-xr-x   - root supergroup          0 2020-10-11 01:40 gutenberg-output8
drwxr-xr-x   - root supergroup          0 2020-10-11 01:56 gutenberg-output9
drwxr-xr-x   - root supergroup          0 2020-10-07 19:48 input
drwxr-xr-x   - root supergroup          0 2020-10-08 19:57 output
```

In [13]:
!ls

base		       docker-compose.yml  Makefile  nodemanager      submit
datanode	       hadoop.env	   namenode  README.md
docker-compose-v3.yml  historyserver	   nginx     resourcemanager


In [14]:
%cd ..
!ls

/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs
docker-hadoop				     hadoop_source
hadoop-mapreduce-examples-2.7.1-sources.jar  python_hadoop_examples


In [15]:
!docker ps

CONTAINER ID        IMAGE                             COMMAND                  CREATED             STATUS                 PORTS                    NAMES
3c3fb342cbe9        sakha002/nodemanager:latest       "/entrypoint.sh /run…"   3 hours ago         Up 3 hours (healthy)   8042/tcp                 nodemanager1
4964d37233e1        sakha002/resourcemanager:latest   "/entrypoint.sh /run…"   3 hours ago         Up 3 hours (healthy)   0.0.0.0:8089->8088/tcp   resourcemanager
29f240e07a9a        sakha002/historyserver:latest     "/entrypoint.sh /run…"   3 hours ago         Up 3 hours (healthy)   8188/tcp                 historyserver
4842dd9e1b7b        sakha002/datanode:latest          "/entrypoint.sh /run…"   3 hours ago         Up 3 hours (healthy)   9864/tcp                 datanode1
0e4d8cbf0bb5        sakha002/datanode:latest          "/entrypoint.sh /run…"   3 hours ago         Up 3 hours (healthy)   9864/tcp                 datanode2
1e452e8cb7ab        sakha002/datanode:latest     

In [17]:
!docker cp hadoop_source/hadoop-streaming-3.1.0.jar  namenode:.

In [22]:
!ls python_hadoop_examples/example3/


mapper.py  reducer.py


In [24]:
!docker cp python_hadoop_examples/example3/.   namenode:media/


okay so here a docker command got me stuck for a few. Note that here I wanted to copy all files within the example3 directory to copy in my destination path.

I first tried  example3/  which did not work and also tried  example3/* and it is not working, so after a few searches I dfound this :

https://stackoverflow.com/questions/32566624/docker-cp-all-files-from-a-folder-to-existing-container-folder

Okay now I have all my files there. Only need to run the streaming command:


```
hadoop jar hadoop-*streaming*.jar \
-mapper /media/mapper.py \
-reducer /media/reducer.py \
-input /root/gutenberg/*  \
-output /root/gutenberg-output20 

```

okay so looks like a few errors:


first:
```
hadoop jar hadoop-*streaming*.jar \
-mapper /media/mapper.py \
-reducer /media/reducer.py \
-input gutenberg/*  \
-output gutenberg-output20 

```
second looks like my python mapper reducer fall into error, so need to make them execuatable.
Okay so I made two chages 

 
```
chmod +x mapper.py 
```
and for the other one and then changing this:
```
#!/usr/bin/python3
```
and finally the command should be like this
```
hadoop jar hadoop-*streaming*.jar -file /media/mapper.py -mapper /media/mapper.py -file /media/reducer.py -reducer /media/reducer.py -input gutenberg/*  -output gutenberg-output23 
```


## Another Example:



In [26]:
!ls
%cd python_hadoop_examples/
%mkdir example4
%cd example4

docker-hadoop				     hadoop_source
hadoop-mapreduce-examples-2.7.1-sources.jar  python_hadoop_examples
/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs/python_hadoop_examples
/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs/python_hadoop_examples/example4


In [27]:
!git clone https://github.com/eljefe6a/nfldata.git


Cloning into 'nfldata'...
remote: Enumerating objects: 660, done.
remote: Total 660 (delta 0), reused 0 (delta 0), pack-reused 660
Receiving objects: 100% (660/660), 17.05 MiB | 9.33 MiB/s, done.
Resolving deltas: 100% (353/353), done.
Updating files: 100% (54/54), done.


In [30]:
%cd nfldata/
!cat stadiums.csv
!dos2unix -l -n stadiums.csv unixstadiums.csv
!cat unixstadiums.csv # Hooray! One stadium per line

[Errno 2] No such file or directory: 'nfldata/'
/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs/python_hadoop_examples/example4/nfldata
University of Phoenix Stadium,63400,78600,Glendale Arizona,419 Tifway Bermuda Grass,FALSE,ARI,2006,GHCND:USW00023183,Retractable,1150dos2unix: converting file stadiums.csv to file unixstadiums.csv in Unix format...
University of Phoenix Stadium,63400,78600,Glendale Arizona,419 Tifway Bermuda Grass,FALSE,ARI,2006,GHCND:USW00023183,Retractable,1150

```
hdfs dfs -mkdir nfldata/
hdfs dfs -mkdir nfldata/stadiums
hdfs dfs -put /media/unixstadiums.csv  nfldata/stadiums/

hadoop jar hadoop-*streaming*.jar \
    -input nfldata/stadiums \
    -output nfldata/output1 \
    -mapper cat \
    -reducer "wc -l"

# now we check our results:
hadoop fs -ls nfldata/output1

# looks like files are there, lets get the result:
hadoop fs -text nfldata/output1/part*
# => 32

```


In [31]:
!ls

173328.csv		  drivestransform.py	      README.md
adddriveresult.hql	  input			      setup.sh
adddrives.hql		  license.md		      spark
arrests.csv		  playbyplay_join.hql	      src
columns.txt		  playbyplay_tablecreate.hql  stadiums.csv
conf			  queries.hql		      unixstadiums.csv
drivesresulttransform.py  queries.pig


In [32]:
!docker cp unixstadiums.csv namenode:media/unixstadiums.csv

```
#mapper
import sys

for line in sys.stdin:
    line = line.strip()
    unpacked = line.split(",")
    stadium, capacity, expanded, location, surface, turf, team, opened, weather, roof, elevation = line.split(",")
    results = [turf, "1"]
    print("\t".join(results))


```

okay so in order to get myself a bit familar on how line.split unpack, etc. works, I changed it to this which is really not that big a deal


```
with open ("./unixstadiums.csv") as file:

    lines= file.readlines()
    for line in lines:
    
        line = line.strip()
        unpacked = line.split(",")
        stadium, capacity, expanded, location, surface, turf, team, opened, weather, roof, elevation = line.split(",")
        results = [turf, "1"]
        print("\t".join(results))`)
```

```
#reducer

import sys

# Example input (ordered by key)
# FALSE 1
# FALSE 1
# TRUE 1
# TRUE 1
# UNKNOWN 1
# UNKNOWN 1

# keys come grouped together
# so we need to keep track of state a little bit
# thus when the key changes (turf), we need to reset
# our counter, and write out the count we've accumulated

last_turf = None
turf_count = 0

for line in sys.stdin:

    line = line.strip()
    turf, count = line.split("\t")

    count = int(count)
    # if this is the first iteration
    if not last_turf:
        last_turf = turf

    # if they're the same, log it
    if turf == last_turf:
        turf_count += count
    else:
        # state change (previous line was k=x, this line is k=y)
        result = [last_turf, turf_count]
        print("\t".join(str(v) for v in result))
        last_turf = turf
        turf_count = 1

# this is to catch the final counts after all records have been received.
print("\t".join(str(v) for v in [last_turf, turf_count]))

```

In [39]:
!ls
%cd gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs/python_hadoop_examples/example4
!ls

 gdrive			      Hossein_personal	'System Volume Information'
 homepage_extracted_fromuce   HOSSEIN_WCGEC	 Tempfiles
/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs/python_hadoop_examples/example4
mapper.py  reducer.py  unixstadiums.csv


In [40]:
!ls

mapper.py  reducer.py  unixstadiums.csv


In [41]:
%cd ..

/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/hadoop_n_docker/my_dfs/python_hadoop_examples


In [43]:
!docker cp ./example4/. namenode:media/

```
hadoop jar hadoop-*streaming*.jar \
    -file media/mapper.py \
    -mapper media/mapper.py \
    -file media/reducer.py \
    -reducer media/reducer.py \
    -input nfldata/stadiums \
    -output nfldata/pythonoutput \   

```

```
hadoop fs -text nfldata/pythonoutput/part-*
FALSE 15
TRUE 17
```